<a href="https://colab.research.google.com/github/mnansary/matParticleSizeAnalyzer/blob/master/colab_gen_UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# colab specific task
*   mount google drive
*   change working directory to git repo
*   Check TF version
*    TPU check



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


In [6]:
cd /content/gdrive/My\ Drive/PROJECTS/F2O/pyF2O

/content/gdrive/My Drive/PROJECTS/F2O/pyF2O


In [7]:
!pip3 install tensorflow==1.13.1

## TPU Check

In [8]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

tf.__version__

TPU address is grpc://10.64.59.26:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 2166915970125125229),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2585489592008126641),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 599828549275468201),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 15343725044248921498),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4337914720234411030),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 8385752892503373613),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 18367955200504105097),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 6524066464040497915),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 6073493431776260

'1.13.1'

# UNET generator Model Training

## Compile Model

In [0]:
from tensorflow.keras.optimizers import Adam
from models.generators import UNET
model=UNET()
model.summary()
model.compile(optimizer=Adam(lr=0.01), loss=tf.keras.losses.binary_crossentropy)


## Convert Model

In [0]:
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

def convert_model_TPU(model):
  return tf.contrib.tpu.keras_to_tpu_model(model,strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

model=convert_model_TPU(model)

## Training Parameters

In [0]:
from sklearn.model_selection import train_test_split
epochs=250
batch_size=30
Xt,Xv,Yt,Yv = train_test_split(X,Ys, test_size=0.2)
print(Xt.shape)
print(Xv.shape)
print(Yt.shape)
print(Yv.shape)

(537, 256, 256, 3)
(135, 256, 256, 3)
(537, 256, 256, 1)
(135, 256, 256, 1)


## Train

In [0]:
history=model.fit(Xt,Yt,validation_data=(Xv,Yv),epochs=epochs,batch_size=batch_size, verbose=1)

## Save Model Weights

In [0]:
model.save_weights('sim_net.h5')

## Plot Training Histoty

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('LOSS')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')